In [0]:
from pyspark.sql import functions as F
from delta.tables import DeltaTable
from datetime import datetime

# Step 1 – Read config file
config_path = "/Volumes/practice/default/practice/config.csv"
config_df = spark.read.option("header", "true").csv(config_path).collect()[0]

file_path = config_df["loca"]
stage_table = config_df["tablename1"]
raw_table = config_df["tablename2"]

# Step 2 – Read source data from file path
src_df = spark.read.option("header", "true").csv(file_path)

# Step 3 – Load into Stage Table (truncate + insert)
src_df.write.mode("overwrite").saveAsTable(stage_table)

# Step 4 – Prepare for merge into Raw Table
current_date = datetime.now().strftime("%d/%m/%Y")

stage_df = src_df.withColumn("last_updated_date", F.lit(current_date))

# Step 5 – Merge Stage into Raw Table
if spark._jsparkSession.catalog().tableExists(raw_table):
    raw_delta = DeltaTable.forName(spark, raw_table)
    raw_delta.alias("raw").merge(
        stage_df.alias("stg"),
        "raw.employee_id = stg.employee_id"
    ).whenMatchedUpdateAll(
        set={
            "name": "stg.name",
            "department": "stg.department",
            "salary": "stg.salary",
            "join_date": "stg.join_date",
            "city": "stg.city",
            "last_updated_date": F.lit(current_date)
        }
    ).whenNotMatchedInsertAll().execute()
else:
    stage_df.write.mode("overwrite").saveAsTable(raw_table)

print("ETL process completed successfully.")
